Load some packages 

In [1]:
%run preample.ipy

Import specific functions 

In [2]:
from Utilities import *
from BiCM import *

# Numerical Solving BiCM
    - Get degree sequences
    - run numerical simulation to solve system
    - calculate the link probabilities
    

In [3]:
def get_degree_sequences(M_hat):
    
    degree_sequence_item = np.sum(M_hat,1)
    degree_sequence_product = np.sum(M_hat,0)
    degree_sequence = list(np.concatenate([degree_sequence_item, degree_sequence_product]))
    n_rows = len(degree_sequence_item)
    n_cols = len(degree_sequence_product)
    return degree_sequence, n_rows, n_cols

In [4]:
def get_link_probabilities(M, solver = 'root'):
    [degree_sequence, n_rows, n_cols] = get_degree_sequences(M)
    bicm = bicm_leastSquares(degree_sequence, n_rows, n_cols)
    if solver == 'root':
        bicm.solve_root()
    if solver == 'least_squares':
        bicm.solve_least_squares()
    bicm.generate_p_adjacency()
    return bicm.p_adjacency, abs(np.sum(bicm.final_result))

# Load Venezuelan Bank Adjacency

In [5]:
import os
file_dir = '../data/intermediate/bank/adjacency_link_removed/'

for file in tqdm(os.listdir(file_dir)):
    adj = np.loadtxt(file_dir + file, delimiter=',').astype(int)
    
    original_path = '../data/intermediate/bank/adjacency/' + file[0:9] + file[14:20] + '.csv'
    original_matrix = np.loadtxt(original_path, delimiter=',').astype(int)
    num_removed_links = np.sum(original_matrix) - np.sum(adj)
    
    p_adj, cost = get_link_probabilities(adj)
    if cost > 1:
        p_adj, cost = get_link_probabilities(adj, 'least_squares')
    if cost > 1:
        continue
    #assert cost < 1, 'numerical convergence problems'
    df = sort_probabilities(p_adj, adj)
    R = fill_miss_links(df, adj, num_removed_links, 0)
    np.savetxt('../data/intermediate/bank/adjacency_reconstructed/' + file, R)
    np.savetxt('../data/intermediate/bank/probabilities_adjacency/' + file, p_adj)
    


100%|██████████| 425/425 [02:12<00:00,  1.13s/it]
